In [1]:
# to expirment with sample playback

In [2]:
import os, vlc, time, subprocess, re, wave, contextlib, csv, spacy

In [3]:
nlp = spacy.load('en')

In [9]:
# class to handle the Poems
class Poem():
    def __init__(self, first_line):
        """take in the first line which contains the name of the
        poem as well a the path to a recording of the name of the poem"""
        self.title = nlp(first_line[0])
        self.durations = []
        self.rec_paths = [first_line[1]]
        self.text = [first_line[0]]
        self.full_text = ''
        
        # get the authors name from folder name, add space between first and last
        a = os.path.dirname(first_line[1][6:])
        last_name_index = re.search(r'^([^A-Z]*[A-Z]){2}', a).span()[1] - 1
        self.author = a[:last_name_index] + " " + a[last_name_index:]
        
        # for keeping track of the media
        self.media = []
        self.total_duration = None
        self.instance = vlc.Instance()
        
        with contextlib.closing(wave.open(self.rec_paths[0],'r')) as f:
            frames = f.getnframes()
            rate = f.getframerate()
            self.durations.append(frames / float(rate))
            
        self.total_duration = self.calculateTotalDuration(self.durations)
        self.media.append(self.instance.media_new(self.rec_paths[-1]))
    
    def calculateTotalDuration(self, durs):
        total_duration = 0.0
        for dur in durs:
            total_duration = total_duration + dur
        return total_duration
                          
    def loadLine(self, line):
        """load in each individual line in the poem"""
        self.text.append(nlp(line[0]))
        self.full_text = self.full_text + " " + line[0]
        self.rec_paths.append(line[1])
        with contextlib.closing(wave.open(self.rec_paths[-1],'r')) as f:
            frames = f.getnframes()
            rate = f.getframerate()
            self.durations.append(frames / float(rate))
                          
        self.media.append(self.instance.media_new(self.rec_paths[-1]))
        self.total_duration = self.calculateTotalDuration(self.durations)
    
    def printStats(self, verbose=2):
        print("author : ", self.author)
        print("title : ", self.title)
        print("paths : ", self.rec_paths[:verbose])
        print("text : ", self.text[:verbose])
        print("full text : ", self.full_text)
        print("dur  : ", self.durations[:verbose])
        print("total duration : ", self.total_duration)
        print("media : ", self.media[:verbose])

In [10]:
# helper functions from voiceshell.py
def meanv(coords):
    sumv = [0] * len(coords[0])
    for item in coords:
        for i in range(len(item)):
            sumv[i] += item[i]
    mean = [0] * len(sumv)
    for i in range(len(sumv)):
        mean[i] = float(sumv[i]) / len(coords)
    return mean

def cosine(v1, v2):
    if norm(v1) > 0 and norm(v2) > 0:
        return dot(v1, v2) / (norm(v1) * norm(v2))
    else:
        return 0.0

def sentvec(s):
    sent = nlp(s)
    word_vectors = [w.vector for w in sent]
    return meanv(word_vectors)

def spacy_closest_sent(space, input_str, n=1):
    input_vec = sentvec(input_str)
    return sorted(space,
                  key=lambda x: cosine(np.mean([w.vector for w in x], axis=0), input_vec), reverse=True)[:n]

In [11]:
# a class to handle playback
class PlaybackEngine():
    def __init__(self):
        instance = vlc.Instance()
        self.player = instance.media_player_new()

    def playAll(self, poem, verbose=False):
        if type(poem) is list:
            for p in poem:
                if verbose is True:
                    print("----------------------------------------")
                    print(p.author,": ", p.title, " : ", p.total_duration)
                for i, media in enumerate(p.media):
                    if verbose is True:
                        print(p.text[i], " : ", p.durations[i])
                    self.player.set_media(media)
                    self.player.play()
                    time.sleep(p.durations[i])
        elif type(poem) is Poem:
            self.playTitle(poem)
            if verbose is True:
                print(poem.author, ": ", poem.title, " : ", poem.total_duration)
            for i, media in enumerate(poem.media):
                if verbose is True:
                    print(poem.text[i])
                self.player.set_media(media)
                self.player.play()
                time.sleep(poem.durations[i])
        else:
            print("Please pass a Poem object into the play function")
    
    def playPart(self, poem, part, verbose=False):
        if verbose is True:
            print(poem.author,": ", poem.title, 
                  " : line number ", part, " : ", poem.text[part])
        self.player.set_media(poem.media[part])
        self.player.play()
        time.sleep(poem.durations[part])
        
    def playTitle(self, poem, verbose=False):
        if verbose is True:
            print("poet: ", poem.author," : ", poem.title)
        self.player.set_media(poem.media[0])
        self.player.play()
        time.sleep(poem.durations[0])

In [12]:
# create a playback engine
pe = PlaybackEngine()

In [13]:
# time to get the poem text
csv_file = 'voiceshell_audio_LUT.csv'
def loadCSV(csv_file):
    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        all_lines = list(reader)

    poems = []
    last_author = None
    current_author = None
    for line in all_lines:
        last_author = current_author
        current_author = os.path.dirname(line[1][6:])
        if current_author == "":
            pass
            # print("blank line :  ", line)
        elif current_author != last_author:
            if len(poems) > 0:
                pass
                # poems[-1].printStats()
            poems.append(Poem(line))
        else:
            poems[-1].loadLine(line)
    return poems
    
poems = loadCSV(csv_file)

In [14]:
# can use the printStats method to get info about each poem
# poems[1].printStats()

In [15]:
print(len(poems))

31


In [16]:
# play through all the poems one at a time in order
pe.playAll(poems[1:], verbose=True)

----------------------------------------
Angela Andrews :  White Saris  :  48.37877551020408
White Saris  :  2.034081632653061
What I knew of their house was a blue garage door, kicked-in at the middle.  :  6.326485260770975
A buckling dent.  :  2.2381632653061225
You approach a scene like that with caution.  :  3.5781632653061224
It takes time to know whether to pause or hurry, head down.  :  5.404081632653061
Their glowing white saris on this grey marble day.  :  4.904081632653061
Outside the buckled door, a station wagon, black.  :  5.274081632653061
I was thinking of you. How you will find out.  :  4.168163265306123
The black suit you will wear.  :  2.304081632653061
How the feel of your best white shirt could hurt.  :  3.804081632653061
How silk might hang in a cold wardrobe.  :  3.7740816326530613
I pushed you through those people.   :  2.804081632653061
I pushed you along the road.  :  1.7651473922902494
----------------------------------------
Tusiata Avia :  Shower  :  109.219

KeyboardInterrupt: 

In [365]:
# play a random single poem in its entirety
pe.playAll(poems[random.randint(0, len(poems))], verbose=True)

Bob Orr :  Eternity  :  106.45938775510203
Eternity
Eternity is the traffic lights at Huntly before they change from red to green.
I am lost in the enchantment of an ancient entertainment.
A wraith-like old wooden two-storey hotel a war memorial hall with a padlocked front door.
the sour taste on my tongue of a dust and diesel railway station.
miners cottages pale as mushrooms in the mist.
a seesaw in the playground of a primary school.
like scales that tilt toward injustice.
all become fantastical and floating like some surreal craft now cut adrift by phantom boatmen.
Tilted on the river’s broad traverse the topsy-turvy of its history down a surface cross threaded and riddled with mysteries wide from its flashing underbelly its streetlights like a gorse bloom’s yellow carnival through coal black waters voyaging this corridor of stars.
Do I merely chance to catch a glimpse of Mum and Dad after a day out at the races waltzing on the balcony of the Waipa Hotel.
Dad with his pockets full 

In [17]:
# play all the poem titles
for poem in poems:
    pe.playTitle(poem, verbose=True)

poet:  Amy Brown  :  The Propaganda Poster Girl
poet:  Angela Andrews  :  White Saris
poet:  Tusiata Avia  :  Shower
poet:  Marty Smith  :  AGNUS DEI
poet:  Sonja Yelich  :  and-yellow
poet:  Anna Jackson  :  Spring
poet:  Anna Livesey  :  Shoeman in Love
poet:  Ashleigh Young  :  Certain Trees
poet:  Bernadette Hall  :  The History of Europe
poet:  Bob Orr  :  Eternity
poet:  Brian Turner  :  Fear
poet:  Chris Price  :  Rose and fell.
poet:  Dinah Hawken  :  365 x 30.
poet:  Elizabeth Smither  :  TWO SECURITY GUARDS TALKING ABOUT JUPITER.
poet:  Frances Samuel  :  LIFE-DRAWING CLASS.
poet:  Gordon Challis  :  Walking an imaginary dog.
poet:  Graham Lindsay  :  big bed.
poet:  Ian Wedde  :  To Death.
poet:  Jennifer Compton  :  THE THREEPENNY KOWHAI STAMP BROOCH
poet:  Joan Flemming  :  THEORY OF LIGHT
poet:  John Dennison  :  LONE KAURI (REPRISE).
poet:  John Gallas  :  THE MONGOLIAN WOMEN'S ORCHESTRA.
poet:  Kay McKenzieCooke  :  SACRED DAYS.
poet:  Kerrin PSharpe  :  LIKE RAIN THE T

In [18]:
# play a random line from a poem
for i in range(0,10):
    poem_index = random.randint(0, len(poems))
    section_index = random.randint(0, len(poems[poem_index].text))
    # print(poems[_index].printStats())
    pe.playTitle(poems[poem_index])
    pe.playPart(poems[poem_index], section_index, verbose=True)

Graham Lindsay :  big bed.  : line number  2  :  Sleepy baby with the goldfish lips, deep dark lashes, angel-pink cheeks, ears like truffles, or hatchcovers for underground shelters.


KeyboardInterrupt: 

In [23]:
# set up the voice recognition port of the program
import speech_recognition as sr
import pyaudio

sr = sr.Recognizer()
with sr.Microphone() as source:
    print("Calibrating microphone for ambient noise... (this will take 5 seconds)")
    r.adjust_for_ambient_noise(source, duration=5)

AttributeError: 'Recognizer' object has no attribute 'Microphone'

In [5]:
#!usr/bin/env python  
#coding=utf-8  

import pyaudio  
import wave  

#define stream chunk   
chunk = 1024  

#open a wav format music  
f = wave.open(r"/home/pi/voiceshell/Audio/AmyBrown/AmyBrown10.wav","rb")  
#instantiate PyAudio  
p = pyaudio.PyAudio()  
#open stream  
stream = p.open(format = p.get_format_from_width(f.getsampwidth()),  
                channels = f.getnchannels(),  
                rate = f.getframerate(),  
                output = True)  
#read data  
data = f.readframes(chunk)  

#play stream  
while data:  
    stream.write(data)  
    data = f.readframes(chunk)  

#stop stream  
stream.stop_stream()  
stream.close()  

#close PyAudio  
p.terminate()  

FileNotFoundError: [Errno 2] No such file or directory: '/home/pi/voiceshell/Audio/AmyBrown/AmyBrown10.wav'